In [1]:
import re
import os
import pickle
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from nltk import ngrams
from collections import Counter
import random
from markov_autocomplete.autocomplete import Autocomplete

In [ ]:
nationality = ' United Kingdom '
engReviews = hotelReviews.loc[hotelReviews['Reviewer_Nationality'] == nationality][['Positive_Review','Negative_Review']]
allReviews = list(engReviews['Positive_Review'].append(engReviews['Negative_Review']))

In [ ]:
def preprocess(rev, stop_words = stopwords.words('English'), punctuations="""!"#$%&\'()*+,./:;<=>?@[\\]^_{|}~"""):
    rev = re.sub('[' + punctuations + ']', ' ', rev).strip()
    s = [item for sublist in [rev.lower().split()] for item in sublist]
    s = [x for x in s if x not in stop_words]
    s = ' '.join(s)
    return s

In [ ]:
processedReviews = []
i = 0
for ar in allReviews:
    if i % (len(allReviews) / 10) == 0:
        print 100.0 * i / len(allReviews)
    i+=1
    processedReviews.append(preprocess(ar))

In [ ]:
random.shuffle(processedReviews)
train = processedReviews[100:]
test = processedReviews[:100]

In [ ]:
with open('processedReviews.pkl', 'wb') as f:
    pickle.dump(processedReviews, f)

with open('processedTrain.pkl', 'wb') as f:
    pickle.dump(train, f)
    
with open('processedTest.pkl', 'wb') as f:
    pickle.dump(test, f)

In [ ]:
with open('processedReviews.pkl', 'rb') as f:
    processedReviews = pickle.load(f)
    
with open('processedTrain.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('processedTest.pkl', 'rb') as f:
    test = pickle.load(f)

In [2]:
def getTestValues(sentences, grams = 1):

    testValues = []

    for s in sentences:
        if grams == 1:
            oneGrams = ac.get_ngrams(s,1)
            for o in range(len(oneGrams) - 1):
                testValues.append([oneGrams[o], oneGrams[o+1]])
        if grams == 2:
            twoGrams = ac.get_ngrams(s,2)
            for o in range(len(twoGrams) - 2):
                testValues.append([twoGrams[o], twoGrams[o+1].split()[-1]])
        if grams == 3:
            threeGrams = ac.get_ngrams(s,3)
            for o in range(len(threeGrams) - 3):
                testValues.append([threeGrams[o], threeGrams[o+1].split()[-1]])

    return testValues

In [6]:
with open('processedTrain.pkl', 'rb') as f:
    train = pickle.load(f)
    
with open('processedTest.pkl', 'rb') as f:
    test = pickle.load(f)

ac = Autocomplete(model_path="ngram_traintest10",
                  sentences=train,
                  n_model=3,
                  n_candidates=20,
                  match_model="start",
                  min_freq=0,
                  punctuations="",
                  lowercase=True)


testValues = getTestValues(test, 3)
predictionSuccess = []
index = 0
for t in testValues:
    if index % (len(testValues) / 200) == 0:
        print 100.0 * index / len(testValues)
    index+=1
    
    p,v = ac.predictions(t[0])
    pPos = [i for i in range(len(p)) if t[1] in p[i]]
    if(len(pPos) == 0):
        predictionSuccess.append(-1)
    else:
        predictionSuccess.append(pPos[0])

0.0
0.470114170584
0.940228341169
1.41034251175
1.88045668234
2.35057085292
2.82068502351
3.29079919409
3.76091336467
4.23102753526
4.70114170584
5.17125587643
5.64137004701
6.1114842176
6.58159838818
7.05171255876
7.52182672935
7.99194089993
8.46205507052
8.9321692411
9.40228341169
9.87239758227
10.3425117529
10.8126259234
11.282740094
11.7528542646
12.2229684352
12.6930826058
13.1631967764
13.6333109469
14.1034251175
14.5735392881
15.0436534587
15.5137676293
15.9838817999
16.4539959704
16.924110141
17.3942243116
17.8643384822
18.3344526528
18.8045668234
19.274680994
19.7447951645
20.2149093351
20.6850235057
21.1551376763
21.6252518469
22.0953660175
22.565480188
23.0355943586
23.5057085292
23.9758226998
24.4459368704
24.916051041
25.3861652116
25.8562793821
26.3263935527
26.7965077233
27.2666218939
27.7367360645
28.2068502351
28.6769644056
29.1470785762
29.6171927468
30.0873069174
30.557421088
31.0275352586
31.4976494291
31.9677635997
32.4378777703
32.9079919409
33.3781061115
33.84822

In [8]:
1.0*sum(np.array(predictionSuccess) != -1) / len(predictionSuccess)

0.25050369375419745

In [7]:
np.save('ThreeGrams_TRTE10',predictionSuccess)

In [ ]:
testsentences = ['''What an amazing experience with really over the top friendly staff!\
                This was one of my first stays with Marriott and they really did everything right.\
                Special bedding orders - everything was arranged.\
                Room size wishes - I was upgraded.\
                The room service was fast and the portion really big with reasonable prices.\
                No need to go somewhere else after a stressful day!\
                Standard decent Marriott hotel.\
                The staff are efficient and the room we had was really quite good - large, clean and comfortable beds.\
                The hotel bar - Champions - is the standard American type bar that Marriotts have.\
                The wait staff at the bar were very nice.\
                An emblematic hotel in the center of Athens.\
                This will take into account the meetings that take place in this hotel.\
                It is a great hotel with excellent modern decoration and clean spaces.\
                The staff is courteous and willing to serve you all with a smile.\
                The meeting rooms are quite large with comfortable seating and a good acoustic.\
                The food is very delicious and it is definitely worth visiting for a meal or a stay.\
                Probably one of the top hotels in Athens, with a top roof bar (Galaxy bar) overlooking the Acropolis and Lycabetus hills.\
                The hotel offers a number of restaurants and boasts an impressive outdoor swimming pool.\
                A great choice to stay while in Athens or visit for drinks or dinner.\
                What amazing views from the Terrace Suite at Four Seasons New York.\
                The staff were all friendly, courteous, and discrete. The hotel lobby, bar, and lounge were beautiful and welcoming.\
                The room in which I stayed, had spectacular views.\
                I will look forward to many returns at this particular Four Seasons Hotel, New York.\
                Super nice staff and rooms are not bad.\
                Redone suite was super nice.\
                Club room has drinks and good snacks.\
                Hotel works as a stop for a night.\
                Not sure I'd book if you want to be in the heart of Munich.\
                It does have shops and restaurants around and good access to mass transit.\
                Room was clean, and was set up nicely.\
                Pool was really nice, and the view from the pool was fantastic.\
                At this point though, not having free wifi in the rooms (even though I'm a member of the club) is completely unacceptable.\
                This is my favorite hotel when visiting London.\
                The accommodations are excellent and the staff is very friendly and efficient.\
                The location is prime with superb views and adjacent parks for walking.\
                Oh yes a near by resistance is Buckingham Palace !!\
                Superior place , enjoyed our stay to the max , the location is excellent , just few minutes walking  to public transportation , there are quite nice good food restaurants and coffee shops very near the hotel ، Lebanese and Turkish restaurants too.\
                Hotel staff and service is great and is clean.\
                Only problem is the decor in lobby and restaurant area.\
                Instead of walking into a hotel in Greece, I felt like I was walking into a hotel in China.\
                I would prefer the greek experience when in Greece.\
                They do book a very large amount of Asians.\
                I guess money talks.\
                This hotel had a nice breakfast and clean rooms.\
                Staff was also helpful.\
                The pool closes at 7.\
                Wonderful bakery across the street!''']